In [1]:
import sys

sys.path.append("../")

import pandas as pd
import numpy as np
import datetime
import os
from pprint import pprint
import matplotlib.pyplot as plt
import time
import vectorbtpro as vbt
from time import time
import helpers as pth
import platform
from dotenv import load_dotenv
import scipy.stats as stats
import time
import helpers as pth
from numba import njit
import talib

theme = "light"
vbt.settings.set_theme(theme)
# optional
# vbt.settings["plotting"]["layout"]["width"] = 640
# vbt.settings["plotting"]["layout"]["height"] = 320

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 20)
# plt.rcParams["axes.grid"] = True
plt.rcParams["figure.figsize"] = (12, 7)
plt.rcParams["axes.formatter.useoffset"] = False
plt.rcParams["axes.formatter.limits"] = [-1000000000, 1000000000]
plt.style.use("classic" if theme == "light" else "dark_background")

if platform.system().lower() == "windows":
    base_data_path = "H:\\phitech-data\\01_raw"
else:
    from core_chains.simple.llm import make_Q_chain

    base_data_path = "../../phitech-data/01_raw"
    load_dotenv("../../sandatasci-core/credentials")
    Q = make_Q_chain("gpt-4o-instance1", __vsc_ipynb_file__)

In [2]:
start = "2024-10-01"
end = "2024-10-10"
symbols = ["MES"]
chunk_size = 2

chunks = pth.get_date_chunks(start, end, chunk_size=1)
chunks

[('2024-10-01', '2024-10-02'),
 ('2024-10-02', '2024-10-03'),
 ('2024-10-03', '2024-10-04'),
 ('2024-10-04', '2024-10-05'),
 ('2024-10-05', '2024-10-06'),
 ('2024-10-06', '2024-10-07'),
 ('2024-10-07', '2024-10-08'),
 ('2024-10-08', '2024-10-09'),
 ('2024-10-09', '2024-10-10')]

In [3]:
chunk_start, chunk_end = chunks[0][0], chunks[1][1]

df = pth.SierraChartData.pull(symbols, start=chunk_start, end=chunk_end)
df = df.resample("1min")
df.shape

(2880, 1)

### Simple stop loss

In [ ]:
from collections import namedtuple


Memory = namedtuple("Memory", ["counter"])
init_memory = lambda target_shape: Memory(
    counter=np.full(target_shape[1], np.nan)
)


@njit(nogil=True)
def signal_func(c, m):
    if c.i % 10 == 0:
        return pth.LONG_ENTER()
    
    if c.i % 15 == 0:
        return pth.LONG_EXIT()

    position =  c.last_pos_info[c.col]
    if position['status'] == vbt.pf_enums.TradeStatus.Open:
        sl_info = c.last_sl_info[c.col]
        if not vbt.pf_nb.is_stop_info_active_nb(sl_info):
            if vbt.pf_nb.in_long_position_nb(c):
                entry_price = position['entry_price']
                vbt.pf_nb.set_sl_info_nb(
                    sl_info,
                    init_price=entry_price,
                    init_idx=c.i,
                    stop=entry_price - 1.0,
                    delta_format=vbt.pf_enums.DeltaFormat.Target,
                )

    return pth.NO_SIGNAL()


pf = vbt.Portfolio.from_signals(
    df.get('Close').iloc[:100],
    signal_func_nb=signal_func,
    signal_args=(
        vbt.RepFunc(init_memory),
    ),
    size=1.0,
    size_granularity=1,
    size_type=vbt.pf_enums.SizeType.Percent,
    init_cash=100000,
    # jitted=False,
)
pf.plot_value()

In [ ]:
pf.orders.readable

,Order Id,Column,Signal Index,Creation Index,Fill Index,Size,Price,Fees,Side,Type,Stop Type
0,0,MES,2024-10-01 00:00:00+00:00,2024-10-01 00:00:00+00:00,2024-10-01 00:00:00+00:00,16.0,5929.0,0.0,Buy,Market,None
1,1,MES,2024-10-01 00:01:00+00:00,2024-10-01 00:01:00+00:00,2024-10-01 00:01:00+00:00,16.0,5928.0,0.0,Sell,Market,SL
2,2,MES,2024-10-01 00:10:00+00:00,2024-10-01 00:10:00+00:00,2024-10-01 00:10:00+00:00,16.0,5928.0,0.0,Buy,Market,None
3,3,MES,2024-10-01 00:11:00+00:00,2024-10-01 00:26:00+00:00,2024-10-01 00:26:00+00:00,16.0,5927.0,0.0,Sell,Market,SL
4,4,MES,2024-10-01 00:30:00+00:00,2024-10-01 00:30:00+00:00,2024-10-01 00:30:00+00:00,16.0,5927.0,0.0,Buy,Market,None
